In [ ]:
! pip install -U pip
! pip install -U torch==1.5.0
! pip install -U torchaudio==0.5.0
! pip install -U matplotlib==3.2.1
! pip install -U trains>=0.15.0
! pip install -U tensorboard==2.2.1

In [ ]:
import os
import torch
import torchaudio
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from trains import Task

%matplotlib inline

In [ ]:
task = Task.init(project_name='Audio Example', task_name='data pre-processing')
configuration_dict = {'number_of_smaples': 3}
configuration_dict = task.connect(configuration_dict)  # enabling configuration override by trains
print(configuration_dict)  # printing actual configuration (after override in remote mode)

In [ ]:
tensorboard_writer = SummaryWriter('./tensorboard_logs')

In [ ]:
if not os.path.isdir('./data'):
    os.mkdir('./data')
yesno_data = torchaudio.datasets.YESNO('./data', download=True)

In [ ]:
def plot_signal(signal, title, cmap=None):
    plt.figure()
    if signal.ndim == 1:
        plt.plot(signal)
    else:
        plt.imshow(signal, cmap=cmap)    
    plt.title(title)
    plt.show()

In [ ]:
for n in range(configuration_dict.get('number_of_smaples', 3)):
    waveform, sample_rate, labels = yesno_data[n]
    melspectogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate)
    plot_signal(waveform[0,:], 'Original waveform')
    tensorboard_writer.add_audio('Audio samples/{}'.format(n), waveform, n, sample_rate)
    plot_signal(melspectogram_transform(waveform.squeeze()).numpy(), 'Mel spectogram', 'hot')
    plot_signal(torchaudio.transforms.AmplitudeToDB()(melspectogram_transform(waveform.squeeze())).numpy(), 'Mel spectogram DB', 'hot')